<a href="https://colab.research.google.com/github/ToonWeyens/cometml/blob/main/Image_Classification_with_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Comet

In [ ]:
!pip install comet_ml --quiet

# Import Comet

In [ ]:
import comet_ml
comet_ml.init(project_name='keras-image-classification')

COMET INFO: Comet API key is valid


# Import Dependencies

In [ ]:
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Create an Experiment

In [ ]:
#create an experiment with your api key
experiment = comet_ml.Experiment()

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/content' and lookings in parents. You can override where Comet is looking for a Git Patch by setting the configuration `COMET_GIT_DIRECTORY`
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/team-comet-ml/keras-image-classification/208e9262fa564a3f868618ec8a208d94



# Load Data

In [ ]:
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

60000 train samples
10000 test samples


# Define Training Parameters

In [ ]:
# these will all get logged
params = {
    "batch_size": 128,
    "epochs": 2,
    "layer1_type": "Dense",
    "layer1_num_nodes": 64,
    "layer1_activation": 'relu',
    "optimizer": 'adam',
}

num_classes = 10

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)



# Define Model

In [ ]:
model = Sequential()
model.add(Dense(64, activation="relu", input_shape=(784,)))
model.add(Dense(num_classes, activation="softmax"))

# print model.summary() to preserve automatically in `Output` tab
print(model.summary())

model.compile(
    loss="categorical_crossentropy", optimizer=params['optimizer'], metrics=["accuracy"]
)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 64)                50240     
_________________________________________________________________
dense_5 (Dense)              (None, 10)                650       
Total params: 50,890
Trainable params: 50,890
Non-trainable params: 0
_________________________________________________________________
None


# Log Experiment Parameters

In [ ]:
experiment.log_parameters(params)
experiment.log_dataset_hash(x_train)  # creates and logs a hash of your data

# Log Training

In [ ]:
# will log metrics with the prefix 'train_'
model.fit(
    x_train,
    y_train,
    batch_size=params['batch_size'],
    epochs=params['epochs'],
    verbose=1,
    validation_data=(x_test, y_test),
)

Epoch 1/2
469/469 [==============================] - 2s 4ms/step - loss: 0.4220 - accuracy: 0.8840 - val_loss: 0.2295 - val_accuracy: 0.9332
Epoch 2/2
469/469 [==============================] - 2s 3ms/step - loss: 0.2049 - accuracy: 0.9420 - val_loss: 0.1753 - val_accuracy: 0.9481


# Log Evaluation

## Logging Evaluation Metrics

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test)
metrics = {"loss": loss, "accuracy": accuracy}
with experiment.test():
  experiment.log_metrics(metrics)

313/313 [==============================] - 0s 1ms/step - loss: 0.1753 - accuracy: 0.9481


## Logging the Confusion Matrix and Images 

Comet gives you the option to log Images with the `experiment.log_image` method. We're going to use this method along with our Confusion Matrix so that we can log samples from our dataset and identify misclassified images in the UI.   

In [ ]:
predictions = model.predict(x_test)

In [ ]:
# Logs the image corresponding to the model prediction 
def index_to_example(index):
    image_array = x_test[index].reshape(28, 28)
    image_name = "confusion-matrix-%05d.png" % index
    results = experiment.log_image(image_array, name=image_name)
    # Return sample, assetId (index is added automatically)
    return {"sample": image_name, "assetId": results["imageId"]}

LABELS = [f"class_{i}" for i in range(10)]

experiment.log_confusion_matrix(
    y_test[:100],
    predictions[:100],
    labels=LABELS,
    index_to_example_function=index_to_example,
    title="Confusion Matrix: Evaluation",
    file_name="confusion-matrix-eval.json",
)

{'api': 'https://www.comet.ml/api/rest/v2/experiment/asset/get-asset?assetId=06d2a2369fe04380921bb63eb73bb8fc&experimentKey=208e9262fa564a3f868618ec8a208d94',
 'assetId': '06d2a2369fe04380921bb63eb73bb8fc',
 'web': 'https://www.comet.ml/api/asset/download?assetId=06d2a2369fe04380921bb63eb73bb8fc&experimentKey=208e9262fa564a3f868618ec8a208d94'}

# Ending an Experiment

In [ ]:
experiment.end()

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/team-comet-ml/keras-image-classification/208e9262fa564a3f868618ec8a208d94
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [2]                 : (0.8839666843414307, 0.9420166611671448)
COMET INFO:     batch_accuracy [94]          : (0.1015625, 0.9418383836746216)
COMET INFO:     batch_loss [94]              : (0.20554016530513763, 2.3517816066741943)
COMET INFO:     epoch_duration [2]           : (1.5461168570000154, 2.1004652270003135)
COMET INFO:     loss [2]                     : (0.20493067800998688, 0.42200857400894165)
COMET INFO:     test_accuracy                : 0.9480999708175659
COMET INFO:     test_loss                    : 0.17532573640346527
COMET INFO:     val_accuracy [2]             : (0.9332000017166138, 0.9480